In [14]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

In [5]:
bc = datasets.load_breast_cancer()

In [11]:
X = pd.DataFrame(bc.data, columns=bc.feature_names)
Y = bc.target

In [12]:
X.head(2)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902


In [23]:
Xsub = X[['mean symmetry', 'worst radius']]
Xsub = (Xsub - Xsub.mean()) / Xsub.std()

lr = LogisticRegression()

lr.fit(Xsub, Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [24]:
# Get coefs for predictors
print lr.coef_
print lr.intercept_

[[-1.08904585 -4.69834081]]
[ 0.45678836]


In [28]:
# predicted probabliities for classes at Y index 100

y100_pp = lr.predict_proba(Xsub.iloc[100,:])[0]

print y100_pp

[ 0.36331398  0.63668602]


/Users/sebozek/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [31]:
# Do this manually:

X100 = Xsub.iloc[100,:].values
print X100

intercept = lr.intercept_
coefs = lr.coef_[0]

[-0.73909882  0.14913597]


In [33]:
X100_lo = lr.intercept_ + X100[0]*coefs[0] + X100[1]*coefs[1]
print X100_lo

[ 0.56100922]


In [35]:
print np.exp(X100_lo)/(1 + np.exp(X100_lo))

[ 0.63668602]


# 4. Interpret individual coefficients & impact on probability

In [39]:
def log_odds_to_prob(odd):
    return np.exp(odd) / (1+ np.exp(odd))

In [41]:
intercept_p = log_odds_to_prob(intercept)
print intercept_p

[ 0.61225201]


In [38]:
# what if mean sym was 1 std dev than the mean
# but worst radius is the mean?

print coefs

[-1.08904585 -4.69834081]


In [42]:
prob_1std_ms = log_odds_to_prob(intercept + coefs[0] * 1 + coefs[1] * 0)
print prob_1std_ms

[ 0.34699883]


In [44]:
# logodds of different values turned into probabilities
print 'probability of logodds 1:', log_odds_to_prob(1)

# coefficient of 0 indicates equal odds for that predictor - no effect on prob
# no matter predictor value, multiplied by 0, no effect

print 'probability of logodds 1:', log_odds_to_prob(0) 

 probability of logodds 1: 0.73105857863
probability of logodds 1: 0.5


In [47]:
# What is the change in odds of a predictor based on its coefficient?
# Worst radius has big negative coefficient
# If one std dev higher worst radius, what is ∆P of having cancer
# Here 1 is NOT having cancer

my_worst_radius_increase = 1
change_prob_having_cancer_1std_worst_radius = (1- (log_odds_to_prob(coefs[1]*my_worst_radius_increase)))
print change_prob_having_cancer_1std_worst_radius

my_mean_symmetry_increase = 1
change_prob_mean_symmetry = (1- (log_odds_to_prob(coefs[0]*my_mean_symmetry_increase)))
print change_prob_mean_symmetry

0.990971869282
0.748202006074
